In [ ]:
%pylab inline
import os
import os.path as op
import nibabel as nb
from glob import glob
from IPython.display import Image
from nipype import MapNode, Workflow, Node
from nipype.interfaces.freesurfer import MRIConvert
from nipype.interfaces.io import DataSink
from nipype.interfaces.utility import IdentityInterface, Function
from IPython.display import Image
import pandas as pd
from pathlib import Path
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('max_colwidth',500)

### Get intersection of niftis, metadata, and freesurfer, and non-callibration output

In [ ]:
niftis_dir = Path('/data/Dnude/bids_work/nifti_converted_images')
archive_exts = ['.nii.gz','.tgz']
bids_info_pklz = Path('bids_info.pklz')

df_nii = pd.DataFrame(
    {'nifti_path' :
     [p.as_posix() for p in niftis_dir.glob('**/*.nii.gz')]}
)

df_nii.nifti_path[0]

exts_regex = '(?:' + '|'.join(archive_exts) + ')'

df_nii = pd.concat([df_nii,
                       df_nii.nifti_path.str.extract(
                              '.*/(?P<mask_id>[0-9]{2,5})/.*' + exts_regex,expand=True)],
                       axis = 1)

# Filter original ge4x

df_nii = df_nii.loc[~df_nii.nifti_path.str.contains('orig'),:]
df_nii.head()

In [ ]:
len(df_nii)

In [ ]:
df_metadata = pd.read_csv('/data/Dnude/bids_work/metadata/AdamT_NV_data_exchange_Batch01.csv')
df_metadata = df_metadata.assign(
    ScanID=lambda df:df.ScanID.astype(int).astype(str))
df_metadata.head()

In [ ]:
df_merge = df_nii.merge(df_metadata,how = 'outer',indicator=True, left_on= 'mask_id', right_on= 'ScanID' )

df_merge.query("_merge != 'both'")

In [ ]:
df_merge_filtered = df_merge.query("_merge == 'both'").drop(labels = '_merge',axis = 1)

len(df_merge_filtered)

#### Freesurfer output

In [ ]:
df_fs = pd.DataFrame({'fs_id_path' : [f.as_posix() for f in Path('/data/Dnude/subjects').glob('*')]})

df_fs = pd.concat([df_fs,
                       df_fs.fs_id_path.str.extract(
                              '.*/(?P<fs_id>[0-9]{2,5})',expand=True)],
                       axis = 1).assign(
mask_id = lambda df:[str(int(x.split('/')[-1])) for x in df.fs_id])
df_fs.head()

In [ ]:
df_full = df_fs.merge(df_merge_filtered,on = 'mask_id',how = 'outer', indicator = True)
len(df_full.query("_merge != 'both'"))

In [ ]:
df_full.head()

In [ ]:
df_full_filtered = (df_full.
                    query("_merge == 'both'").
                    drop(labels = '_merge',axis = 1)
                   )
df_full_filtered = df_full_filtered.loc[~df_full_filtered.CalibrationStudies.str.contains('not for public release'),:]
df_full_filtered = df_full_filtered.sort_values(['MRN','DateScanText','mask_id'])
len(df_full_filtered)

# Generate bids mapping

In [ ]:
sub_key_path = Path('sub_key.pklz')
def add_bids_ses(df):
    df = df.assign(bids_ses = ['{num:05d}'.format(num = 1 + i) for i in range(len(df))])
    return df

generate_keys = True
if generate_keys:
    def gen_bids_subj(row,sub_key_path,id_col = 'patient_id'):
        if sub_key_path.exists():
            sub_dict = pd.read_pickle(sub_key_path)
            sub = sub_dict.get(row[id_col],None)
            if not sub:
                sub_dict[row[id_col]] = '{n:04d}'.format(n = 1 + max([int(s) for s in sub_dict.values]))
        else:
            sub_dict = pd.Series()
            sub = '0001'
            sub_dict[row[id_col]] = sub
            
        
        
        pd.to_pickle(sub_dict,path=sub_key_path)

        row['bids_subj'] = sub
        return row    
else:
    def gen_bids_subj(row,sub_key_path):
        sub_dict = pd.read_csv('test',sep=' ',dtype=str,header=None,names = ['mrn','sub_id'])
        mrn = row.sub_id
        sub = sub_dict.query('mrn ==@mrn').sub_id.values[0]
        if not sub:
            raise ValueError('sub key does not exist')
        row['bids_subj'] = sub
        return row    

df_bids = df_full_filtered
df_bids = df_bids.apply(lambda row: gen_bids_subj(row,sub_key_path,'MRN'),axis = 1)
df_bids['bids_ses'] = df_bids.fs_id

In [ ]:
df_bids.head()

# Generate symlinked freesurfer_dir with bids mapping

In [ ]:
# linked_fs_dir = Path('linked_bids_2018_08_14') Incorrect
linked_fs_dir = Path('linked_freesurfer_maskid')
linked_fs_dir.absolute()

In [ ]:
if not linked_fs_dir.exists():
    linked_fs_dir.mkdir()

In [ ]:
df_bids.head()

In [ ]:
for row in df_bids.itertuples():
    sub_ses = 'sub-' + row.bids_subj + '_ses-' + row.bids_ses
    !cd {linked_fs_dir.absolute()};ln -s {row.fs_id_path} {sub_ses}

# Setting up Mindcontrol to QC Freesurfer Outputs

This tutorial will set up a Mindcontrol instance for Freesurfer output QC. The first step is to clone the Mindcontrol repository from GitHub and checkout the FreeSurfer branch:

```bash
git clone https://github.com/akeshavan/mindcontrol
cd mindcontrol
git checkout origin/FreeSurfer
```

Then start Mindcontrol by typing `meteor` in the mindcontrol directory:

```bash
meteor
```

Also, start the static server in another terminal window:

```bash
cd mindcontrol_docs/mindcontrol_base_dir
python start_static_server.py
```

Navigate to http://localhost:3000, you should see an empty Mindcontrol UI:

![](images/freesurfer_empty_instance.png)

## Import FreeSurfer Subjects Directory Into Mindcontrol

Freesurfer writes data in `.mgz` format, but Mindcontrol's Papaya.js viewer can only read `.nii.gz` files. We also want to split up the different parts of FreeSurfer QC (brain mask editing, wm edits, and aparc+aseg edits) into 3 sections, as shown in the empty Mindcontrol instance above. This workflow reads all the folders in the `SUBJECTS_DIR` of Freesurfer, converts all volumetric data from `.mgz` into `.nii.gz`, and parses stats files to display as histograms on the mindcontrol interface.

Lets say your directory structure looks like:

```bash
mindcontrol/ #(cloned from https://github.com/akeshavan/mindcontrol, checked out origin/FreeSurfer branch)

mindcontrol_docs/ #(cloned from https://github.com/akeshavan/mindcontrol_docs)
|___ MindPrepFS.ipynb #(this notebook)
|___ mindcontrol_base_dir/
     |___ start_static_server.py

freesurfer_subjects_dir/
|___ sub-01/
    |___ mri/
    |___ stats/
    |___ surf/
     ...
 ...
```

### Import necessary packages

We recommend installing nipype through [Anaconda](https://www.continuum.io/downloads):

```bash
conda config --add channels conda-forge
conda install -y nipype
```

Also make sure you have [FreeSurfer](https://www.google.com/webhp?sourceid=chrome-instant&ion=1&espv=2&ie=UTF-8#q=freesurfer+download&*) installed

# Run conversion for mindcontrol

### Set your directories

In [ ]:
# Set SUBJECTS_DIR, from Freesurfer
subjects_dir = linked_fs_dir.absolute()

# Set the directory to save converted Freesurfer outputs (ideally, the mindcontrol base dir)
mindcontrol_base_dir = os.path.abspath("./mindcontrol_base_dir") #this is where the static_server runs
mindcontrol_outdir = os.path.abspath("./mindcontrol_base_dir/freesurfer") 
if not os.path.exists(mindcontrol_outdir):
    os.mkdir(mindcontrol_outdir)

# Set the directory where the nipype workflow saves intermediary files
workflow_working_dir = os.path.abspath("/lscratch")

After running this worflow, your directory structure will look like:

```bash
mindcontrol/ 
# you should be running "meteor" in a separate terminal window here

mindcontrol_docs/ 
|___ MindPrepFS.ipynb 
|___ mindcontrol_base_dir/
     |___ start_static_server.py
          # you should be running start_static_server.py in a separate terminal window
     |___ freesurfer/ #the workflow will create these files
         |___ sub-01/
             |___ T1.nii.gz
             |___ aparc+aseg.nii.gz
             |___ brainmask.nii.gz
             |___ wm.nii.gz
             |___ mindcontrol_entries.json
|___ scratch/ #this is the nipype working directory

freesurfer_subjects_dir/
|___ sub-01/
    |___ mri/
    |___ stats/
    |___ surf/
     ...
 ...
```

### Find all subjects in the Freesurfer directory

In [ ]:
subject_paths = glob(op.join(subjects_dir, "*"))
# SUBJECTS_DIR sometimes contains directories that aren't subjects, don't grab them
subjects = []
for path in subject_paths:
    subject = path.split('/')[-1]
    # check if mri dir exists, and don't add fsaverage
    if op.exists(op.join(path, 'mri')) and subject != 'fsaverage':
        subjects.append(subject)
        # Volumes that will be prepared
volumes = ["brainmask.mgz", "wm.mgz", "aparc+aseg.mgz", "T1.mgz"]

In [ ]:
# df_mriqc = pd.read_csv('/data/Dnude/bids_work/synced_analysis_directory/anatMRIQC.csv')
# df_armin = pd.read_csv('/data/Dnude/bids_work/metadata/Armin_Science_1531_MASKIDs.csv')

### Helper functions

In [ ]:
#this function finds data in the subjects_dir 
def data_grabber(subjects_dir, subject, volumes):
    import os
    volumes_list = [os.path.join(subjects_dir, subject, 'mri', volume) for volume in volumes]
    return volumes_list

In [ ]:
#this function parses the aseg.stats, lh.aparc.stats and rh.aparc.stats and returns a dictionary

def parse_stats(subjects_dir, subject):
    from os.path import join, exists

    aseg_file = join(subjects_dir, subject, "stats", "aseg.stats")
    lh_aparc = join(subjects_dir, subject, "stats", "lh.aparc.stats")
    rh_aparc = join(subjects_dir, subject, "stats", "rh.aparc.stats")

    assert exists(aseg_file), "aseg file does not exists for %s" %subject
    assert exists(lh_aparc), "lh aparc file does not exists for %s" %subject
    assert exists(rh_aparc), "rh aparc file does not exists for %s" %subject

    def convert_stats_to_json(aseg_file, lh_aparc, rh_aparc):
        import pandas as pd
        import numpy as np

        def extract_other_vals_from_aseg(f):
            value_labels = ["EstimatedTotalIntraCranialVol",
                              "Mask",
                              "TotalGray",
                              "SubCortGray",
                              "Cortex",
                              "CerebralWhiteMatter",
                              "CorticalWhiteMatterVol",
                           "SurfaceHoles"]
            value_labels = list(map(lambda x: 'Measure ' + x + ',', value_labels))
            output = pd.DataFrame()
            with open(f,"r") as q:
                out = q.readlines()
                relevant_entries = [x for x in out if any(v in x for v in value_labels)]
                for val in relevant_entries:
                    sname= val.split(",")[1][1:]
                    vol = val.split(",")[-2]
                    output = output.append(pd.Series({"StructName":sname,"Volume_mm3":vol}),ignore_index=True)
            return output

        df = pd.DataFrame(np.genfromtxt(aseg_file,dtype=str),columns=["Index",
                                                            "SegId",
                                                            "NVoxels",
                                                            "Volume_mm3",
                                                            "StructName",
                                                            "normMean",
                                                            "normStdDev",
                                                            "normMin",
                                                            "normMax",
                                                            "normRange"])

        df = df.append(extract_other_vals_from_aseg(aseg_file), ignore_index=True)
        
        aparc_columns = ["StructName", "NumVert", "SurfArea", "GrayVol",
                         "ThickAvg", "ThickStd", "MeanCurv", "GausCurv",
                         "FoldInd", "CurvInd"]
        tmp_lh = pd.DataFrame(np.genfromtxt(lh_aparc,dtype=str),columns=aparc_columns)
        tmp_lh["StructName"] = "lh_"+tmp_lh["StructName"]
        tmp_rh = pd.DataFrame(np.genfromtxt(rh_aparc,dtype=str),columns=aparc_columns)
        tmp_rh["StructName"] = "rh_"+tmp_rh["StructName"]

        aseg_melt = pd.melt(df[["StructName","Volume_mm3"]], id_vars=["StructName"])
        aseg_melt.rename(columns={"StructName": "name"},inplace=True)
        aseg_melt["value"] = aseg_melt["value"].astype(float)
        
        lh_aparc_melt = pd.melt(tmp_lh,id_vars=["StructName"])
        lh_aparc_melt["value"] = lh_aparc_melt["value"].astype(float)
        lh_aparc_melt["name"] = lh_aparc_melt["StructName"]+ "_"+lh_aparc_melt["variable"]
        
        rh_aparc_melt = pd.melt(tmp_rh, id_vars=["StructName"])
        rh_aparc_melt["value"] = rh_aparc_melt["value"].astype(float)
        rh_aparc_melt["name"] = rh_aparc_melt["StructName"]+ "_"+rh_aparc_melt["variable"]
        
        output = aseg_melt[["name",
                            "value"]].append(lh_aparc_melt[["name",
                                                            "value"]],
                                             ignore_index=True).append(rh_aparc_melt[["name",
                                                                                      "value"]],
                                                                       ignore_index=True)
        outdict = output.to_dict(orient="records")
        final_dict = {}
        for pair in outdict:
            final_dict[pair["name"]] = pair["value"]
        return final_dict
    
    output_dict = convert_stats_to_json(aseg_file, lh_aparc, rh_aparc)
    return output_dict
    

In [ ]:
# This function creates valid Mindcontrol entries that are saved as .json files. 
# They can be loaded into the Mindcontrol database later

def create_mindcontrol_entries(mindcontrol_base_dir, output_dir, subject, stats):
    import os
    from nipype.utils.filemanip import save_json
    
    metric_split = {"brainmask": ["eTIV", "CortexVol", "TotalGrayVol"],
                    "wm": ["WM-hypointensities", 
                           "Right-WM-hypointensities","Left-WM-hypointensities"],
                    "aparcaseg":[]}
    
    volumes = ["brainmask.nii.gz", "wm.nii.gz", "aparc+aseg.nii.gz", "T1.nii.gz"]
    volumes_list = [os.path.join(output_dir, subject, volume) for volume in volumes]

    all_entries = []
    
    for idx, entry_type in enumerate(["brainmask", "wm", "aparcaseg"]):
        entry = {"entry_type":entry_type, 
                 "subject_id": subject, 
                 "name": subject}
        base_img = os.path.relpath(volumes_list[-1], mindcontrol_base_dir)
        overlay_img = os.path.relpath(volumes_list[idx], mindcontrol_base_dir)
        entry["check_masks"] = [base_img, overlay_img]
        entry["metrics"] = {}
        for metric_name in metric_split[entry_type]:
            entry["metrics"][metric_name] = stats.pop(metric_name)
        if not len(metric_split[entry_type]):
            entry["metrics"] = stats
        all_entries.append(entry)
    
    output_json = os.path.abspath("mindcontrol_entries.json")
    save_json(output_json, all_entries)
    return output_json

### Nodes

Define the nipype nodes below:

In [ ]:
input_node = Node(IdentityInterface(fields=['subject_id',"subjects_dir",
                                            "mindcontrol_base_dir", "output_dir"]), name='inputnode')
input_node.iterables=("subject_id", subjects)
input_node.inputs.subjects_dir = subjects_dir
input_node.inputs.mindcontrol_base_dir = mindcontrol_base_dir #this is where start_static_server is running
input_node.inputs.output_dir = mindcontrol_outdir #this is in the freesurfer/ directory under the base_dir

dg_node=Node(Function(input_names=["subjects_dir", "subject", "volumes"],
                      output_names=["volume_paths"], 
                      function=data_grabber), 
             name="datagrab")
#dg_node.inputs.subjects_dir = subjects_dir
dg_node.inputs.volumes = volumes


mriconvert_node = MapNode(MRIConvert(out_type="niigz"), 
                          iterfield=["in_file"], 
                          name='convert')

get_stats_node = Node(Function(input_names=["subjects_dir", "subject"],
                               output_names = ["output_dict"],
                               function=parse_stats), name="get_freesurfer_stats")

write_mindcontrol_entries = Node(Function(input_names = ["mindcontrol_base_dir",
                                                         "output_dir",
                                                         "subject",
                                                         "stats"],
                                          output_names=["output_json"],
                                          function=create_mindcontrol_entries), 
                                 name="get_mindcontrol_entries")

datasink_node = Node(DataSink(),
                     name='datasink')
subst = [('out_file',''),('_subject_id_',''),('_out','')]  + [("_convert%d" % index, "") for index in range(len(volumes))] 
datasink_node.inputs.substitutions = subst

### Workflow

Connect the nodes of the workflow and run it:

In [ ]:
wf = Workflow(name="MindPrepFS")
wf.base_dir = workflow_working_dir
wf.connect(input_node,"subject_id", dg_node,"subject")
wf.connect(input_node,"subjects_dir", dg_node, "subjects_dir")
wf.connect(input_node, "subject_id", get_stats_node, "subject")
wf.connect(input_node, "subjects_dir", get_stats_node, "subjects_dir")
wf.connect(input_node, "subject_id", write_mindcontrol_entries, "subject")
wf.connect(input_node, "mindcontrol_base_dir", write_mindcontrol_entries, "mindcontrol_base_dir")
wf.connect(input_node, "output_dir", write_mindcontrol_entries, "output_dir")
wf.connect(get_stats_node, "output_dict", write_mindcontrol_entries, "stats")
wf.connect(input_node, "output_dir", datasink_node, "base_directory")
wf.connect(dg_node,"volume_paths", mriconvert_node, "in_file")
wf.connect(mriconvert_node,'out_file',datasink_node,'out_file')
wf.connect(write_mindcontrol_entries, "output_json", datasink_node, "out_file.@json")
wf.write_graph(graph2use='exec')
output = wf.run(plugin='MultiProc')

List the files in one of the subjects'd folders to make sure everything is there:

In [ ]:
os.listdir(os.path.join(mindcontrol_outdir,subjects[0]))

Make a graph of our workflow:

In [ ]:
Image(os.path.join(workflow_working_dir, 'MindPrepFS/graph_detailed.dot.png'))

## Load JSON entries into the Mindcontrol database:

In [ ]:
def get_collection(port=7081):
    from pymongo import MongoClient
    client = MongoClient("localhost", port)
    db =  client.meteor
    collection = db.subjects
    return collection, client

In [ ]:
dnude_metadata_pklz = Path('giedd_metadata.pklz')

In [ ]:
df_metadata = pd.read_pickle(dnude_metadata_pklz)
df_metadata = df_metadata.assign(sub_sess = 'sub-' + df_metadata.bids_subj + '_ses-' + df_metadata.bids_ses)
msk = df_metadata.dtypes == np.float  # or object, etc.
df_metadata.loc[:, msk] = df_metadata.loc[:, msk].applymap(float)
df_metadata.head()

In [ ]:
df_mriqc = df_metadata.loc[:,'cjv':'sub_sess']

In [ ]:
df_meta_manual = pd.concat(
    [df_metadata.loc[:,"ScanFormat": 'Post_MNI_CIVET_QC_categorical'].drop(axis = 1, labels = ['Fam','FamID','MRN','LN','FN']),
    df_metadata.loc[:,"sub_sess"]],
    axis = 1
)

## The following cell was run on a host with a running meteor instance and mongo server

In [ ]:
#import a json file reader
from nipype.utils.filemanip import load_json

#get the mongo database collection
coll, cli = get_collection(7081)

#for subject's entry file, load the data and insert or update it into the database
for entry_file in glob(os.path.join(mindcontrol_outdir,"*","mindcontrol_entries.json")):
    data = load_json(entry_file)
    for d in data: 
        name = d['name']
        
        ## additionally add an entry for mriqc
        if entry_type =='brainmask':
            mriqc_entry = d
            if '_id' in mriqc_entry.keys():
                mriqc_entry.pop('_id')
            mriqc_entry['entry_type']  = "mriqc_entry"
            metrics = (df_mriqc.
                                      query("sub_sess == @name").
                                      drop(axis = 1,
                                          labels = ['sub_sess'])
                                     )
            if len(metrics) != 1:
                print('skipping: ', entry_file)
                break
            mriqc_entry['metrics'] = metrics.iloc[0,:].to_dict()
            query = {"name": mriqc_entry['name'], "entry_type": mriqc_entry['entry_type']}
            res = coll.find_one(query)
            #if the entry for a subject/entry_type does not exist, insert it:
            if not res:
                coll.insert_one(mriqc_entry)
            #if the entry exists, simply update it
            else:
                coll.update_one(query, {"$set": mriqc_entry})

        if entry_type =='brainmask':
            manual_meta = d
            if '_id' in manual_meta.keys():
                manual_meta.pop('_id')
            manual_meta['entry_type']  = "manual_meta"
            metrics = (df_meta_manual.
                                      query('sub_sess == @name').
                                      drop(axis = 1,
                                          labels = ['sub_sess'])
                                     )
            assert(len(metrics) == 1)
            manual_meta['metrics'] = metrics.iloc[0,:].to_dict()
            query = {"name": manual_meta['name'], "entry_type": manual_meta['entry_type']}
            res = coll.find_one(query)
            #if the entry for a subject/entry_type does not exist, insert it:
            if not res:
                coll.insert_one(manual_meta)
            #if the entry exists, simply update it
            else:
                coll.update_one(query, {"$set": manual_meta})

        # add original entries
        name, entry_type = d["name"], d["entry_type"]

        query = {"name": name, "entry_type": entry_type}
        res = coll.find_one(query)
        #if the entry for a subject/entry_type does not exist, insert it:
        if not res:
            coll.insert_one(d)
        #if the entry exists, simply update it
        else:
            coll.update_one(query, {"$set": d})

Navigate to http://localhost:3000, You should now see something like:
    
![](images/freesurfer_instance.png)

## Applying painter edits to files

Open the QC viewer of a Brainmask file (click on an entry in the Freesurfer ID column), sign in, and click on the Painter pencil icon. I zoomed in (using alt+left click and drag) and erased an arbitrary chunk of voxels of the brainmask, and also logged a point at the center of my erased blob (will be used to plot later):

![](images/paint_brainmask.png)

Remember to hit "Save" when you are done. Now we will load the world coordinates that we painted and apply them to the brainmask file:

#### Some more imports

In [ ]:
import nibabel as nib
from os.path import join, exists, split
from dipy.tracking import utils
import numpy as np

#### function to load the nifti data using a database query

In [ ]:
def get_segmentation_mask(db, query,mindcontrol_base_dir):
    cursor = db.find(query)
    results = []
    for item in cursor:
        results.append(item)
    
    assert(len(results) == 1)
    subject = results[0]

    return nib.load(join(mindcontrol_base_dir,subject["check_masks"][1]))

#### Converts world coordinates to numpy matrix indices

In [ ]:
def convert_to_indices(streamline, aff, img):
    
    topoints = lambda x : np.array([[m["x"], m["y"], m["z"]] for m in x["world_coor"]])
    points_orig = topoints(streamline)
    points_nifti_space = list(utils.move_streamlines([points_orig], aff, input_space=aff))[0]
    from dipy.tracking._utils import _to_voxel_coordinates, _mapping_to_voxel
    lin_T, offset = _mapping_to_voxel(aff, None)
    idx = _to_voxel_coordinates(points_orig, lin_T, offset)
    return points_nifti_space, idx

#### Record the indices of points to paint and their new paint value

In [ ]:
def get_points_to_paint(drawing, aff, img): 
    import pandas as pd
    df = pd.DataFrame()
    data = img.get_data()
    for d in drawing:
        if not "paintValue" in d.keys():
            continue
        pv = d["paintValue"]
        if len(d["world_coor"]):
            points_nii_space, trans_points = convert_to_indices(d, aff, img)
            tmp = []
            for i,ni in enumerate(trans_points):
                to_append = {"x": ni[0], "y":ni[1], "z": ni[2], "val": pv}

                #validate affine. 
                if len(d["world_coor"]) == len(d["matrix_coor"]):
                    old_val_client = d["matrix_coor"][i]["old_val"]
                    true_val = data[ni[0], ni[1], ni[2]]
                    assert old_val_client == true_val, "affine mismatch!"

                tmp.append(to_append)
            df = df.append(pd.DataFrame(tmp), ignore_index=True)
    if df.shape[0]:
        df.drop_duplicates(inplace=True)

    return df

#### Final function to read the database for coordinates, load the image data, apply the painter edits, and save the image

In [ ]:
def create_paint_volume(mongo_port, query,mindcontrol_base_dir, outpath):
    coll, cli = get_collection(mongo_port)
    mask_img = get_segmentation_mask(coll,query, mindcontrol_base_dir)
    aff = mask_img.affine

    p1 = get_points_to_paint(coll.find_one(query)["painters"], aff, mask_img)

    data = mask_img.get_data()
    for idx, row in p1.iterrows():
        paint_value = row['val']
        x,y,z = row['x'], row['y'], row['z']
        data[x][y][z] = paint_value

    painted_image = nib.nifti1.Nifti1Image(data,aff,mask_img.header)
    painted_image.to_filename(outpath)
    return outpath

#### Run on sub-02:

In [ ]:
outpath = create_paint_volume(3001, {"subject_id": "sub-02", 
                           "entry_type":"brainmask"},
                    mindcontrol_base_dir,
                   os.path.abspath("./mindcontrol_base_dir/freesurfer/sub-02/brainmask_edited.nii.gz"))

Lets make sure this worked by plotting in nilearn

In [ ]:
entry = coll.find_one({"subject_id": "sub-02", "entry_type":"brainmask"})
idx = entry["loggedPoints"][0]["world_coor"]

In [ ]:
from nilearn.plotting import plot_roi

In [ ]:
fig,ax = subplots(1,1,figsize=(10,10))
plot_roi(outpath, "./mindcontrol_base_dir/freesurfer/sub-02/T1.nii.gz", 
         cut_coords=[idx["y"]], display_mode="y", axes=ax)

# Make BIDS dir

In [ ]:
subjects_dir

In [ ]:
df_bids.head()

In [ ]:
df_bids['fs_t1_path'] = (df_bids.
                         apply(lambda df:
                               (Path(mindcontrol_base_dir) / 'freesurfer' / ('sub-' + df.bids_subj + '_ses-' + df.bids_ses)  / 'T1.nii.gz') ,
                               axis = 1)
                        )
df_bids.fs_t1_path.apply(Path.exists).sum()

In [ ]:
def make_symlink(row, overwrite_previous=False,symlink_path = 'symlink_path', source_path = 'dicom_path'):
    """
    Create symlink to source path
    """
    original_dir = Path.cwd()
    
    sympath = Path(row[symlink_path]).absolute().resolve()
    source_path = Path(row[source_path]).absolute()
    relative_path = os.path.relpath(source_path, sympath.parent)
    print(sympath)
    print(source_path)
    print(relative_path)
    symlinked = []
    if sympath.exists() and not overwrite_previous:
        symlinked = False
        print('Symlink exists already, supply "overwrite_previous" argument\
         if you wish to overwrite it')
    else:
        if sympath.exists():
            os.remove(sympath.absolute())
        if not sympath.absolute().parent.exists():
            os.makedirs(sympath.absolute().parent)
        stdout = os.chdir(sympath.absolute().parent)
#         print("symlink %s to %s with link %s"% (sympath, source_path, os.path.relpath(source_path, sympath.parent)))
        stdout = sympath.symlink_to(
            relative_path)
        symlinked = True
    stdout = os.chdir(original_dir)
    return symlinked




linked_bids_dir = Path('linked_bids_2018_08_14')
if not linked_bids_dir.exists():
    linked_bids_dir.mkdir()

df_bids['bids_nifti_path'] = df_bids.apply(
    lambda row: linked_bids_dir.absolute() / ('sub-' + row.bids_subj) / ('ses-' + row.bids_ses) / 'anat' / ('sub-' + row.bids_subj + '_ses-' + row.bids_ses + '_run-01_T1w.nii.gz'),
    axis = 1)
    
import heudiconv_helpers as hh

df_bids.apply(lambda row: make_symlink(row, overwrite_previous=True,symlink_path= 'bids_nifti_path', source_path = 'nifti_path'),axis = 1)

In [ ]:
!ls linked_bids_2018_08_14/

In [ ]:
df_bids.head()

In [ ]:
df_bids.to_pickle(bids_info_pklz)
df_bids.to_csv(bids_info_pklz.with_suffix('.psv'),index=False,sep='|')